In [285]:
import pandas as pd
import numpy as np
import warnings
import scipy as sp
from datetime import datetime
from IPython.display import display
import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from IPython.core.display import display, HTML
from plotly.subplots import make_subplots
from IPython.display import display
autoscroll_set_on_load = True
autoscroll_starting_threshold = -1
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df = pd.read_excel('data/LIGHT4DEAF_Neurocognition_Avril_2020_1_FULL_ADULTES_20200504.xlsx')

In [286]:
#df = df[df["ID_VISITE"] == 1]
df = df[df["Investigateur"].notnull()]

In [287]:
cardinal = len(df)
cardinal

56

In [288]:
now = datetime.now()
df['Date de naissance du patient'] = pd.to_datetime(df['Date de naissance du patient'], format='%m/%d/%y')
df['Date de naissance du patient'] = df['Date de naissance du patient'].where(df['Date de naissance du patient'] < now, df['Date de naissance du patient'] -  np.timedelta64(100, 'Y')) 
df['age'] = (now - df['Date de naissance du patient']).astype('<m8[Y]')

In [289]:
#Patients adultes : type
df_type1 = df[df["Type"]=='Type 1']
df_type2 = df[df["Type"]=='Type 2']
df_type3 = df[df["Type"]=='Type 3']
df_type_nan = df[df["Type"].isna()]

In [290]:
display(HTML('<h1>L i g h t 4 d e a f</h1>'))
display(HTML('<h2>Neurocognition - WAIS</h2>'))
display(HTML("<h3>Description de l'échantillon</h3>"))
display(HTML("<h4>1. Données : </h4>"))
display(HTML("<p>NB : NaN = donnée manquante</p>"))

In [333]:
cols = df.columns.values.tolist()
cols.index('WAIS')
cols.index('Autre commentaire')
arr = cols[cols.index('WAIS'):cols.index('Autre commentaire')]
arr.insert(0,"ID_SECONDAIRE")
arr.insert(1,"age")
arr.insert(2,"Type")

In [334]:
df_wais = df.loc[:,arr]

In [335]:
df_wais.loc[df_wais['Type']=="Type 2", 'Type_color']="#B3D9FF"
df_wais.loc[df_wais['Type']=="Type 1", 'Type_color']="#33ffc9"
df_wais.loc[df_wais['Type']=="Type 3", 'Type_color']="#FFC433"
df_wais.loc[df_wais['Type'].isna(), 'Type_color']="#C5C5C5"
df_wais['Type'].fillna('Type inconnu', inplace= True)

In [336]:
display(HTML('<p>Nombre de patients ayant effectué le test (WAIS = Oui) : '+str(len(df_wais[df_wais['WAIS']=='Oui']))+'/'+str(cardinal)+' patients</p>'))

In [337]:
df_wais_test = df_wais.loc[:,['age','calcul', 'Note standard Similitudes','Note standard Vocabulaire', 'Note standard Information',
                             'Note standard Cubes','Note standard Matrices', 'Note standard Puzzles visuels',
                              'Note standard Mémoires des chiffres', 'Note standard Arithmétique',
                                'Note standard Symboles','Note standard Code'
                             ]]

In [338]:
columns= list(df_wais_test)
myarr = [['Effectif', 'Moyenne', 'Moy. âge', 'Médiane', 'Min.', 'Max.']]
for i in columns[2:]:
    effectif = len(df_wais[df_wais[i].notna()])
    avg_age = round(df_wais[df_wais[i].notna()]['age'].mean())
    avg = round(df_wais[i].mean(),2)
    median = round(df_wais[i].median(),2)
    max_val = df_wais[i].max()
    min_val = df_wais[i].min()
    myarr.append([effectif, avg, avg_age, median, max_val, min_val])

In [339]:
wais_test =  go.Table(
                header=dict(values=list(df_wais_test.columns[1: ]),
                align='left'),
                cells=dict(values= myarr,align='left'))

In [340]:
fig = go.Figure(data=wais_test)
fig.show()

In [341]:
display(HTML('<h4>2. Tests </h4>'))
display(HTML('<h5>2.1. WAIS ICV</h5>'))

In [342]:
display(HTML("<p>Moyenne d'âge : "+str(round(df_wais[df_wais['Total des notes standards ICV'].notna()]['age'].mean(), 2))+"</p>"))

In [343]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1,6,7,8,9,10,11,12,13]]

In [344]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [345]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [346]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[11]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=2)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [347]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane"}})

In [348]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [349]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [350]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]
                           , df_wais_sub_agg.iloc[:,8]], align='left'))

In [351]:
fig = go.Figure(data=wais_sub)
fig.show()

In [352]:
display(HTML('<h5>2.1. WAIS IRP</h5>'))

In [391]:
display(HTML("<p>Moyenne d'âge : "+str(round(df_wais[df_wais['Total des notes standards IRP'].notna()]['age'].mean(), 2))+"</p>"))

In [412]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1,14, 15, 16, 17, 18, 19, 20, 21]]

In [413]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [397]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [398]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[11]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=2)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [358]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane"}})

In [359]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [360]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [361]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]
                           , df_wais_sub_agg.iloc[:,8]], align='left'))

In [362]:
fig = go.Figure(data=wais_sub)
fig.show()

In [401]:
display(HTML('<h5>2.1. WAIS IMT</h5>'))

In [402]:
display(HTML("<p>Moyenne d'âge : "+str(round(df_wais[df_wais['Total des notes standards IRP'].notna()]['age'].mean(), 2))+"</p>"))

In [418]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1,22, 23, 24, 25, 26, 27, 28]]

In [419]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [422]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [424]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [425]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane"}})

In [426]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [427]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [429]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]], align='left'))

In [430]:
fig = go.Figure(data=wais_sub)
fig.show()

In [440]:
display(HTML('<h5>2.1. WAIS IVT</h5>'))

In [441]:
display(HTML("<p>Moyenne d'âge : "+str(round(df_wais[df_wais['Total des notes standards IVT'].notna()]['age'].mean(), 2))+"</p>"))

In [442]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1, 29, 30, 31, 32, 33, 34, 35]]

In [443]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [445]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [446]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [447]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane"}})

In [448]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [449]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [450]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]], align='left'))

In [451]:
fig = go.Figure(data=wais_sub)
fig.show()

In [452]:
display(HTML('<h4>4. WAIS complet</h4>'))

In [457]:
wais = df_wais[(df_wais['Total des notes standards ICV'].notna()) & (df_wais['Total des notes standards IRP'].notna()) & (df_wais['Total des notes standards IMT'].notna()) & (df_wais['Total des notes standards IVT'].notna())]
                                                                

In [460]:
display(HTML("<p>Nombre de patients ayant effectué tous les test : "+str(len(wais))+"</p>"))